# Introduction
The Dashboard is a graphical user interface which can control the EMERGENT backend from any networked computer. Let's see how it works. First, initialize the test network and run the master in the "Network construction" section of the "Getting started" tutorial. Then, launch the Dashboard:

In [1]:
%gui qt5

In [2]:
%cd /emergent/emergent
%run dashboard/main

C:\emergent\emergent
2019-03-04T13:00:24.353464 dashboard: Serving at localhost::27191.
2019-03-04T13:00:24.522465 dashboard:

INFO:root:New listener at localhost on port 27191.


 Received: {'op': 'update', 'params': 1, 'timestamp': '2019-03-04T13:00:24.503465'}
2019-03-04T13:00:24.558463 dashboard: Received message: {'op': 'connect', 'params': {'addr': 'localhost', 'port': 27190}}
2019-03-04T13:00:24.561464 dashboard:2019-03-04T13:00:24.561464 Sending: dashboard: Sending: {'op': 'update', 'params': 1}
 {'op': 'get', 'target': 'state', 'timestamp': '2019-03-04T13:00:24.560464'}
2019-03-04T13:00:24.564465 dashboard: Received: {'op': 'get', 'value': {'hub': State([('thing', {'X': 0, 'Y': 0})])}, 'timestamp': '2019-03-04T13:00:24.564465'}
2019-03-04T13:00:24.818463 dashboard: Sending: {'op': 'get', 'target': 'settings', 'timestamp': '2019-03-04T13:00:24.818463'}
2019-03-04T13:00:24.819462 dashboard: Received: {'op': 'get', 'value': {'hub': {'thing': {'X': {'min': 0, 'max': 1}, 'Y': {'min': 0, 'max': 1}}}}, 'timestamp': '2019-03-04T13:00:24.819462'}
2019-03-04T13:00:38.929422 dashboard: Sending: {'op': 'set', 'target': 'state', 'value': {'hub': {'thing': {'X': 1.0}

Communication between the master and the dashboard is done through the DashAPI class, which we can access through the P2PNode attached to the Dashboard:

In [6]:
api = dash.p2p.api

The API implements get() and set() methods which can currently be applied to the network state and input ranges. For example, let's retrieve the state from the network tree:

In [8]:
state = api.get('state')
print(state)

State([('hub', {'thing': {'X': '2', 'Y': '0.00'}})])


We can also set the state on the network tree:

In [9]:
api.set('state', {'hub': {'thing': {'X': 1.05}}})
new_state = api.get('state')
print(new_state)

2019-03-04T13:05:59.336412 dashboard: Sending: {'op': 'get', 'target': 'settings', 'timestamp': '2019-03-04T13:05:59.335412'}
2019-03-04T13:05:59.337412 dashboard: Received: {'op': 'get', 'value': {'hub': {'thing': {'X': {'min': 0, 'max': 1}, 'Y': {'min': 0, 'max': 1}}}}, 'timestamp': '2019-03-04T13:05:59.336412'}
State([('hub', {'thing': {'X': '1.05', 'Y': '0.00'}})])


You should see that the value displayed in the network tree has changed. Be careful! The API changes the Dashboard only, not the master! In fact, there's a separate class called API for accessing the master; these two classes work together to keep the Dashboard and the master in sync.

For example, let's look at what happens when you update an input in the network tree:
1. The NodeTree.update_editor() method retrieves the state and calls Dashboard.p2p.set('state', state)
2. The P2P node attached to the master retrieves this message and calls the API.set() method.
3. The API passes the actuate command on to all objects referenced by the state.
4. The API checks the new state with API.get('state') and returns the result to the Dashboard P2P node.

Conversely, let's look at what happens when the master initiates a state change:
1. The Hub.actuate() method distributes state changes to all connected Things.
2. Each Thing, after updating its state, calls network.p2p.send({'op': 'set', 'target': 'state', 'value': state}).
3. The Dashboard receives this message and calls the API.set() method to update the user interface.

To summarize, when a local object is changed, it uses the P2PNode.set() method to send a command over the network; then, the remote P2PNode calls the API.set() method to implement the change.